# Импорт библиотек

In [1]:
from torch import nn
import json
from utils import *
import model
import dataset

import imageio
import scipy
from imageio import imread
import os
import torchvision.transforms as transforms
import numpy as np
import random

from model_training import load_models, validate, train

In [49]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

In [52]:
list_of_references = [['this', 'is', 'bad', 'and', 'not', 'good']]
hypotheses = ['this', 'is', 'bad', 'and', 'not', 'good']
sentence_bleu(list_of_references, hypotheses)

1.0

In [59]:
list_of_references = [[['this', 'is', 'bad', 'and', 'not', 'good']],
                      [['i', 'just', 'want', 'to', 'sleep']]]
hypotheses = [['this', 'is', 'bad', 'and', 'not', 'good'], ['i', 'just', 'want', 'to', 'sleep']]
corpus_bleu(list_of_references, hypotheses)

1.0

In [22]:
list_of_references[0]

[['this', 'is', 'bad']]

In [21]:
hypotheses

[['this', 'is', 'bad'], ['and', 'this', 'sucks']]

In [48]:
hyp1 = ['It', 'is', 'a', 'fucking', 'idiot', 'aaaa', 'aaaa', 'aaaa', 'aaaa']
ref1a = ['It', 'is', 'a', 'idiot', 'aaaa', 'aaaa', 'aaaa', 'aaaa']
hyp2 = ['he', 'read', 'the', 'suck', 'aaaa', 'aaaa', 'aaaa', 'aaaa']
ref2a = ['he', 'read', 'the', 'suck', 'aaaa', 'aaaa', 'aaaa', 'aaaa']
list_of_references = [[ref1a, ref1b, ref1c], [ref2a]]
hypotheses = [hyp1, hyp2]
corpus_bleu(list_of_references, hypotheses)

0.7949160929023952

In [46]:
corpus_bleu(list_of_references, hypotheses)

0.6815612934162696

In [19]:
candidates

[['this', 'is', 'a', 'test']]

In [10]:
score

1.0

In [8]:
corpus_bleu(list_of_references, hypotheses)

C:\Users\Sibmice\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


1.2213386697554703e-77

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def set_seed(seed: int = 666) -> None:
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.set_deterministic(True)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed()

# Создание файлов для обучения

In [ ]:
create_input_files(dataset = 'coco', 
                   karpathy_json_path = 'data/dataset_coco.json', 
                   image_folder = 'data/',
                   output_folder = 'images/', 
                   captions_per_image = 5)

# Загрузка словарей

In [4]:
with open('images/WORDMAP_COCO.json') as f:
    wordmap = json.load(f)

In [5]:
res = dict((v,k) for k,v in wordmap.items())

wordmap - словарь, каждому слову ставящий в соответствие его позиционный индекс\
res - Обратный к wordmap словарь

In [6]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor

unorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))

# Загрузка/инициализация модели

In [7]:
# Параметры модели
encoded_image_size = 14
vocab_size = len(wordmap)
word_embeddings_dim = 512
attention_dim = 1024
decoder_hidden_size = 1024
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint_name = None

In [8]:
start_epoch, end_epoch, loss_fn, enc, dec, optimizer_encoder, optimizer_decoder = load_models(checkpoint_name, 
                                                                                              encoded_image_size, 
                                                                                              word_embeddings_dim, 
                                                                                              attention_dim,
                                                                                              decoder_hidden_size, 
                                                                                              vocab_size, 
                                                                                              device)

# Инициализация загрузчиков данных

In [9]:
batch_size = 100
workers = 0

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
set_seed()
train_loader = torch.utils.data.DataLoader(dataset.CaptionDataset('images/', 
                                                                  'TRAIN', 
                                                                  transform=transforms.Compose([normalize])),
                                           batch_size=batch_size, 
                                           shuffle=True, 
                                           num_workers=workers, 
                                           pin_memory=True)

val_loader = torch.utils.data.DataLoader(dataset.CaptionDataset('images/',  
                                                                'VAL',
                                                                transform=transforms.Compose([normalize])),
                                         batch_size=200, 
                                         shuffle=True, 
                                         num_workers=workers, 
                                         pin_memory=True)

In [ ]:
for epoch in range(start_epoch, end_epoch):
    print('epoch:', epoch)
    
    validate(enc, dec, device, loss_fn, val_loader, wordmap, epoch)
    
    train(enc, dec, device, loss_fn, train_loader, optimizer_decoder, optimizer_encoder, epoch)

epoch: 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124


C:\Users\Sibmice\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\Sibmice\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\Sibmice\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  

Epoch 0, BLEU4 3.610692794087871e-232
Current loss 9.080148696899414
Current loss 9.074583053588867
Current loss 9.064139366149902
Current loss 9.060214042663574
Current loss 9.050524711608887
Current loss 9.047754287719727
Current loss 9.037107467651367
Current loss 9.03409481048584
Current loss 9.029791831970215
Current loss 9.016519546508789
Current loss 9.010162353515625
Current loss 9.006659507751465
Current loss 8.991703987121582
Current loss 8.998209953308105
Current loss 8.979903221130371
Current loss 8.968111038208008
Current loss 8.97008991241455
Current loss 8.970489501953125
Current loss 8.953086853027344
Current loss 8.937758445739746
Current loss 8.926980018615723
Current loss 8.931140899658203
Current loss 8.928882598876953
Current loss 8.926633834838867
Current loss 8.914115905761719
Current loss 8.904156684875488
Current loss 8.8965482711792
Current loss 8.903844833374023
Current loss 8.866510391235352
Current loss 8.863779067993164
Current loss 8.837275505065918
Curre

Current loss 5.660126209259033
Current loss 5.568655490875244
Current loss 5.603954792022705
Current loss 5.640878677368164
Current loss 5.548842906951904
Current loss 5.508049964904785
Current loss 5.544387340545654
Current loss 5.543577671051025
Current loss 5.659160614013672
Current loss 5.6003875732421875
Current loss 5.602065086364746
Current loss 5.669164180755615
Current loss 5.624915599822998
Current loss 5.6495585441589355
Current loss 5.63355016708374
Current loss 5.609461307525635
Current loss 5.412810802459717
Current loss 5.609991550445557
Current loss 5.472652912139893
Current loss 5.51173210144043
Current loss 5.511253833770752
Current loss 5.556321620941162
Current loss 5.517375469207764
Current loss 5.622422695159912
Current loss 5.495642185211182
Current loss 5.517813205718994
Current loss 5.622678279876709
Current loss 5.547423839569092
Current loss 5.554732322692871
Current loss 5.44674015045166
Current loss 5.523104667663574
Current loss 5.387653827667236
Current l

Current loss 5.227961540222168
Current loss 5.322674751281738
Current loss 5.3082051277160645
Current loss 5.303111553192139
Current loss 5.3423590660095215
Current loss 5.335162162780762
Current loss 5.289954662322998
Current loss 5.2791619300842285
Current loss 5.257541656494141
Current loss 5.312283992767334
Current loss 5.308685302734375
Current loss 5.444085597991943
Current loss 5.247443199157715
Current loss 5.216730117797852
Current loss 5.254927158355713
Current loss 5.284293174743652
Current loss 5.204614162445068
Current loss 5.3463945388793945
Current loss 5.320532321929932
Current loss 5.239416122436523
Current loss 5.3265380859375
Current loss 5.181760311126709
Current loss 5.296640872955322
Current loss 5.314542770385742
Current loss 5.34822416305542
Current loss 5.3329339027404785
Current loss 5.274959564208984
Current loss 5.371612548828125
Current loss 5.198740005493164
Current loss 5.394064903259277
Current loss 5.299089431762695
Current loss 5.321537017822266
Curren

...

Current loss 2.863474130630493
Current loss 2.9049525260925293
Current loss 2.824769973754883
Current loss 3.0493502616882324
Current loss 3.058121919631958
Current loss 2.9285929203033447
Current loss 2.9800503253936768
Current loss 2.8633649349212646
Current loss 2.8431813716888428
Current loss 3.0501348972320557
Current loss 2.8263776302337646
Current loss 2.9374783039093018
Current loss 2.857811689376831
Current loss 2.8826417922973633
Current loss 2.983181953430176
Current loss 2.7814691066741943
Current loss 2.9842402935028076
Current loss 2.88217830657959
Current loss 2.8833487033843994
Current loss 2.9415876865386963
Current loss 2.8665108680725098
Current loss 2.931887149810791
Current loss 3.0094802379608154
Current loss 2.851257562637329
Current loss 2.755582332611084
Current loss 2.9821951389312744
Current loss 2.997260570526123
Current loss 2.8628416061401367
Current loss 3.119899272918701
Current loss 2.85447096824646
Current loss 3.0148298740386963
Current loss 2.9836816

Current loss 2.8870415687561035
Current loss 2.8051517009735107
Current loss 2.7114808559417725
Current loss 2.93882417678833
Current loss 2.8944718837738037
Current loss 2.804196357727051
Current loss 2.7790393829345703
Current loss 2.892732620239258
Current loss 2.9155232906341553
Current loss 2.8210184574127197
Current loss 2.8669593334198
Current loss 2.9163033962249756
Current loss 2.874483823776245
Current loss 2.883061408996582
Current loss 2.83414888381958
Current loss 2.8335113525390625
Current loss 3.023953676223755
Current loss 2.894291639328003
Current loss 2.822999954223633
Current loss 2.8695855140686035
Current loss 2.95564603805542
Current loss 3.0417628288269043
Current loss 2.7347564697265625
Current loss 3.0383284091949463
Current loss 3.027836561203003
Current loss 2.8737852573394775
Current loss 2.9946231842041016
Current loss 3.048754930496216
Current loss 2.9839043617248535
Current loss 2.8891520500183105
Current loss 2.7849225997924805
Current loss 2.85333108901

Current loss 2.7736473083496094
Current loss 2.8206069469451904
Current loss 3.0321507453918457
Current loss 2.822091817855835
Current loss 2.823772430419922
Current loss 2.770049571990967
Current loss 3.050239086151123
Current loss 2.992131471633911
Current loss 2.9101192951202393
Current loss 2.8288371562957764
Current loss 3.0385053157806396
Current loss 2.887153387069702
Current loss 2.7858378887176514
Current loss 2.9401605129241943
Current loss 2.903757333755493
Current loss 2.859053373336792
Current loss 2.696333408355713
Current loss 2.9309866428375244
Current loss 2.895305871963501
Current loss 2.9300756454467773
Current loss 3.043358564376831
Current loss 2.9673802852630615
Current loss 2.947413444519043
Current loss 2.691145896911621
Current loss 3.126434087753296
Current loss 2.797211170196533
Current loss 2.987370491027832
Current loss 3.0048859119415283
Current loss 3.008371591567993
Current loss 3.0254063606262207
Current loss 2.928840398788452
Current loss 2.89444446563

Current loss 2.9720005989074707
Current loss 2.8024439811706543
Current loss 3.0394837856292725
Current loss 2.960012674331665
Current loss 2.806607484817505
Current loss 2.8762013912200928
Current loss 2.975451707839966
Current loss 2.9277210235595703
Current loss 2.9229061603546143
Current loss 2.7857723236083984
Current loss 2.788630247116089
Current loss 2.8695592880249023
Current loss 2.773693323135376
Current loss 2.970545768737793
Current loss 2.901399612426758
Current loss 2.8631837368011475
Current loss 3.0258049964904785
Current loss 2.898968458175659
Current loss 2.85939359664917
Current loss 2.993321418762207
Current loss 2.8683459758758545
Current loss 2.848238468170166
Current loss 2.8804478645324707
Current loss 2.8564019203186035
Current loss 2.963179588317871
Current loss 2.869966983795166
Current loss 2.841491222381592
Current loss 2.974508047103882
Current loss 2.8990237712860107
Current loss 2.966250419616699
Current loss 2.85493540763855
Current loss 2.917310953140

Current loss 2.9850704669952393
Current loss 2.967047929763794
Current loss 2.917494773864746
Current loss 2.89876389503479
Current loss 2.8661959171295166
Current loss 3.0082507133483887
Current loss 2.918306589126587
Current loss 2.877267360687256
Current loss 3.0055251121520996
Current loss 2.9873032569885254
Current loss 2.8765716552734375
Current loss 2.7144415378570557
Current loss 2.829354763031006
Current loss 2.8152410984039307
Current loss 2.9804818630218506
Current loss 2.9394989013671875
Current loss 3.037201404571533
Current loss 2.895061492919922
Current loss 2.8559212684631348
Current loss 2.8746345043182373
Current loss 2.9849069118499756
Current loss 2.9014413356781006
Current loss 2.927769422531128
Current loss 2.8432321548461914
Current loss 2.9219629764556885
Current loss 2.8908824920654297
Current loss 2.8997132778167725
Current loss 2.8424463272094727
Current loss 2.8995656967163086
Current loss 2.810215473175049
Current loss 2.848020076751709
Current loss 2.86216